# Homework
Baltimore City Crime Data

https://data.baltimorecity.gov/datasets/part1-crime-data/explore?showTable=true
## DataFrames:
1. Specify the schema for the crime data set. 
2. Read the file using the schema definition
3. Cache the DataFrame
4. Show the count of the rows
5. Print the schema
6. Display first 5 rows
7. Answer the following questions using DataFrame operations


In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 53.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=83af688872adc3ecaeb1592f11d865ef4fdf7e276a9392b31047421c2271b124
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [71]:
from pyspark.sql.types import StructType, IntegerType, DateType, FloatType, StringType, StructField
#Columns:X	Y	RowID	CrimeDateTime	CrimeCode	Location	Description	Inside_Outside
#	Weapon	Post	District	Neighborhood	Latitude	Longitude	GeoLocation	Premise	
#VRIName	Total_Incidents	Shape
schema = StructType([
    StructField("X", FloatType()),
    StructField("Y", FloatType()),
    StructField("RowID", IntegerType()),
    StructField("CrimeDateTime", StringType()),
    StructField("CrimeCode", StringType()),
    StructField("Location", StringType()),
    StructField("Description", StringType()),
    StructField("Inside_Outside", StringType()),
    StructField("Weapon", StringType()),
    StructField("Post", StringType()),
    StructField("District", StringType()),
    StructField("Neighborhood", StringType()),
    StructField("Latitude", FloatType()),
    StructField("Longitude", FloatType()),
    StructField("GeoLocation", StringType()),
    StructField("Premise", StringType()),
    StructField("VRIName", StringType()),
    StructField("Total_Incidents", IntegerType()),
    StructField("Shape", IntegerType()),
])
df = spark.read.csv('Part1_Crime_data.csv', header=True, schema=schema)
df.show()
df.printSchema()


+---------+---------+-----+--------------------+---------+--------------------+--------------+--------------+-------+----+---------+--------------------+--------+---------+------------------+---------------+------------+---------------+-----+
|        X|        Y|RowID|       CrimeDateTime|CrimeCode|            Location|   Description|Inside_Outside| Weapon|Post| District|        Neighborhood|Latitude|Longitude|       GeoLocation|        Premise|     VRIName|Total_Incidents|Shape|
+---------+---------+-----+--------------------+---------+--------------------+--------------+--------------+-------+----+---------+--------------------+--------+---------+------------------+---------------+------------+---------------+-----+
|1420074.1| 594160.6|    1|2022/02/26 04:00:...|       4E|   200 W MONUMENT ST|COMMON ASSAULT|          null|     NA| 124|  CENTRAL|        MOUNT VERNON| 39.2975| -76.6193|(39.2975,-76.6193)|           null|        null|              1| null|
|1411374.2| 589791.4|    2|2

In [17]:
df.cache()

DataFrame[X: float, Y: float, RowID: int, CrimeDateTime: string, CrimeCode: string, Location: string, Description: string, Inside_Outside: string, Weapon: string, Post: string, District: string, Neighborhood: string, Latitude: float, Longitude: float, GeoLocation: string, Premise: string, VRIName: string, Total_Incidents: int, Shape: int]

In [22]:
print("There are", df.count(), "rows!")

There are 513158 rows!


In [24]:
df.show(5)

+---------+---------+-----+--------------------+---------+-------------------+--------------+--------------+-------+----+---------+----------------+--------+---------+------------------+---------------+------------+---------------+-----+
|        X|        Y|RowID|       CrimeDateTime|CrimeCode|           Location|   Description|Inside_Outside| Weapon|Post| District|    Neighborhood|Latitude|Longitude|       GeoLocation|        Premise|     VRIName|Total_Incidents|Shape|
+---------+---------+-----+--------------------+---------+-------------------+--------------+--------------+-------+----+---------+----------------+--------+---------+------------------+---------------+------------+---------------+-----+
|1420074.1| 594160.6|    1|2022/02/26 04:00:...|       4E|  200 W MONUMENT ST|COMMON ASSAULT|          null|     NA| 124|  CENTRAL|    MOUNT VERNON| 39.2975| -76.6193|(39.2975,-76.6193)|           null|        null|              1| null|
|1411374.2| 589791.4|    2|2022/02/26 01:26:...|

# Following Questions:
1. What are distinct crime codes?


In [33]:
import pyspark.sql.functions as func
df.select("CrimeCode").distinct().sort("CrimeCode").show()

+---------+
|CrimeCode|
+---------+
|       1A|
|       1K|
|       1O|
|       2A|
|       2B|
|      3AF|
|     3AJF|
|     3AJK|
|     3AJO|
|      3AK|
|      3AO|
|       3B|
|      3BJ|
|       3C|
|      3CF|
|      3CK|
|      3CO|
|       3D|
|       3E|
|      3EF|
+---------+
only showing top 20 rows



2. Count the number of crimes by the crime codes and order by the resulting counts in descending order


In [44]:
df.select("CrimeCode").groupBy("CrimeCode").count().orderBy("count", ascending = False).show(10)

+---------+-----+
|CrimeCode|count|
+---------+-----+
|       4E|91650|
|       6D|68427|
|       5A|43928|
|       7A|40274|
|       6J|27636|
|       6G|26858|
|       6E|24300|
|       6C|23227|
|       4C|22438|
|       5D|14971|
+---------+-----+
only showing top 10 rows



3. Which neighborhood had most crimes?



In [45]:
df.select("Neighborhood").groupBy("Neighborhood").count().orderBy("count", ascending = False).show(1)

+------------+-----+
|Neighborhood|count|
+------------+-----+
|    DOWNTOWN|17799|
+------------+-----+
only showing top 1 row



Downtown had the most

4. Which month of the year had most crimes?


In [74]:
#Converting the string to datetime was tedious and tiring so imma do this instead
df = df.withColumn('Month', func.substring('CrimeDateTime', 6,2))
df.printSchema()
df.show(truncate=False)

root
 |-- X: float (nullable = true)
 |-- Y: float (nullable = true)
 |-- RowID: integer (nullable = true)
 |-- CrimeDateTime: string (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)
 |-- Shape: integer (nullable = true)
 |-- Month: string (nullable = true)

+---------+---------+-----+----------------------+---------+---------------------+--------------+--------------+-------+----+---------+--------------------------------+--------+---------+---------

In [75]:
df.select("Month").groupBy("Month").count().orderBy("count", ascending = False).show(1)

+-----+-----+
|Month|count|
+-----+-----+
|   08|46327|
+-----+-----+
only showing top 1 row



August had the most

5. What weapons were used? 


The following are the various weapons used

In [77]:
df.select("Weapon").distinct().sort("Weapon").show()

+-------+
| Weapon|
+-------+
|   null|
|   FIRE|
|FIREARM|
|  HANDS|
|  KNIFE|
|     NA|
|  OTHER|
+-------+



If this question is asking what weapon was used that month:

In [81]:
df.select("Weapon").where("Month == 08").show()

+------+
|Weapon|
+------+
|    NA|
|    NA|
|    NA|
|    NA|
|    NA|
|    NA|
|    NA|
|    NA|
| OTHER|
|    NA|
|    NA|
|    NA|
|    NA|
| KNIFE|
|    NA|
|    NA|
| OTHER|
| KNIFE|
| KNIFE|
|    NA|
+------+
only showing top 20 rows



6. Which weapon was used the most?

If im assuming, the question means what weapon is used the most:

In [94]:
df.select("Weapon").groupBy("Weapon").count().orderBy("count", ascending = False).collect()[1]
#Have to use collect cause I CANT SUBSCRIPT ANYTHING IN THIS $#%@*^ THING

Row(Weapon='FIREARM', count=46139)

If the Question is refering to what was the most used weapon in aughust

In [95]:
df.select("Weapon").where("Month == 08").groupBy("Weapon").count().orderBy("count", ascending = False).collect()[1]

Row(Weapon='FIREARM', count=4534)